# Training Models

### Setup Data

In [1]:
# basic cnn training
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.utils import multi_gpu_model, to_categorical

import keras
import tensorflow as tf

import numpy as np
import os, sys, errno

import csv
import codecs
import pickle
import h5py

import sys, os, glob
sys.path.append("/home/bryanb/cuckoo-cli/")
import cuckoo_cl

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, num_labels=1, batch_size=32, dim=(32,1024), n_channels=1, n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.num_labels = num_labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.num_labels))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            filename = '/datax/scratch/bbrzycki/training/training2/data/1sig/train/%s.npy' % ID
            X[i] = np.load(filename).reshape((*self.dim, self.n_channels))
            
            X[i] -= np.mean(X[i])
            X[i] /= np.std(X[i])

            # Store class
            y[i] = self.labels[ID]
#         y = to_categorical(y)

        return X, y

# dimensions of our images.
img_width, img_height = 32, 1024
                                                                 
tsamp = 1.4316557653333333

dir_name = '/datax/scratch/bbrzycki/training/training2/'
h5_datasets = '/datax/scratch/bbrzycki/training/training2/data/1sig/1sig.hdf5'
# validation_csv_fn = dir_name + 'train/validation_labels.csv'

batch_size = 64

# Parameters
params = {'dim': (32, 1024),
          'batch_size': batch_size,
          'n_channels': 1,
          'num_labels': 2,
          'shuffle': True}

Using TensorFlow backend.


In [29]:
# Datasets
partition = {'train' : [], 'validation' : []}
labels = {}

total_image_num = 120000
print(total_image_num)
train_num = int(total_image_num * 0.8)
validation_num = total_image_num - train_num

# max_drift = 31.225e-6
dbs = []
ids = []
save_labels = []
with h5py.File(h5_datasets, 'r') as f:
    
    # for 1 signal and 0 rfi
    for i in range(total_image_num):
        ID = '%06d' % i
        
        class_nums = f['train'][ID].attrs['class_nums']
        sig_db = f['train'][ID].attrs['sig_db']
        dbs.append(sig_db)
        
        ids.append(ID)

        start_index, end_index, line_width, snr, class_label = np.squeeze(f['train'][ID]['signals_info'])
        # All between 0 and 1
        label = (start_index / 1024, end_index / 1024)
        labels[ID] = label
        save_labels.append(label)



partition['train'] = ids[:train_num]
partition['validation'] = ids[train_num:]
 
# labels = {key: (value / max_drift) for (key, value) in labels.items()}

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

120000


In [36]:
save_labels = [val for (key, val) in labels.items()]

In [ ]:
np.save('/datax/scratch/bbrzycki/training/training2/data/1sig/dbs.npy', dbs)
np.save('/datax/scratch/bbrzycki/training/training2/data/1sig/labels.npy', save_labels)

Load

In [4]:
total_image_num = 120000
print(total_image_num)
train_num = int(total_image_num * 0.8)
validation_num = total_image_num - train_num

ids = ['%06d' % i for i in range(total_image_num)]
dbs = np.load('/datax/scratch/bbrzycki/training/training2/data/1sig/dbs.npy')
save_labels = np.load('/datax/scratch/bbrzycki/training/training2/data/1sig/labels.npy')
labels = {('%06d' % i): save_labels[i] for i in range(total_image_num)}

# Generators
partition = {'train' : [], 'validation' : []}
partition['train'] = ids[:train_num]
partition['validation'] = ids[train_num:]

120000


In [5]:
batch_size = 64

# Parameters
params = {'dim': (32, 1024),
          'batch_size': batch_size,
          'n_channels': 1,
          'num_labels': 2,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

In [6]:
# keras.backend.clear_session()

# Design model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 1024, 1)))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='linear'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 1022, 32)      320       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 1022, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 1020, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 28, 1020, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 510, 32)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 508, 32)       9248      
_________________________________________________________________
activation_3 (Activation)    (None, 12, 508, 32)       0         
__________

In [ ]:
model_fn = dir_name + 'models/1sig/simple_cnn.h5'
history_fn = dir_name + 'models/1sig/simple_cnn_history'

epochs = 20

history = model.fit_generator(generator=training_generator,
                    steps_per_epoch=train_num // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=validation_num // batch_size,
                    callbacks=[keras.callbacks.ModelCheckpoint(model_fn, monitor='loss', verbose=0, save_best_only=True, mode='auto'), 
                               keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.0001), 
                               keras.callbacks.EarlyStopping(monitor='loss', patience=5,verbose=0, mode='auto')])

model.save_weights(model_fn)
with open(history_fn, 'wb') as f:
    pickle.dump(history.history, f)

# cuckoo_clock.send_email('Training notebook finished!', 'Training in notebook finished')

In [43]:
%matplotlib inline
import matplotlib.pyplot as plt

# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

Signal (db): 20
Label: (343.0, 504.0)
Prediction: (348.7, 508.4)
Sqrt of Error:  7.193618857255543


In [61]:
with h5py.File(h5_datasets, 'r') as f:
    split = 4000
    index = np.random.randint(0, split * 6)
    ID = '%06d' % index
        
    class_nums = f['test'][ID].attrs['class_nums']
    sig_db = f['test'][ID].attrs['sig_db']
    print('Signal (db):', sig_db)

    
    start_index, end_index, line_width, snr, class_label = np.squeeze(f['test'][ID]['signals_info'])
    # All between 0 and 1
    label = (start_index / 1024, end_index / 1024)
    print('Label:', (start_index, end_index))

    
filename = '/datax/scratch/bbrzycki/training/training2/data/1sig/test/%s.npy' % ID
frame = np.load(filename)
X = np.copy(frame).reshape((1, 32, 1024, 1))
X -= np.mean(X)
X /= np.std(X)

p_start, p_end = model.predict(X)[0]
print('Prediction: (%0.1f, %0.1f)' % (p_start * 1024, p_end * 1024))

print('Sqrt of Error (index units): %0.2f' % (1024 * np.sqrt(np.mean((label[0] - p_start)**2 + (label[1] - p_end)**2))))

fig = plt.figure(figsize=(8,6))
plt.imshow(frame, aspect='auto')
plt.plot([p_start * 1024, p_end * 1024], [0, 32], 'w')
plt.colorbar()
plt.show()

plt.imshow(X.reshape(32, 1024), aspect='auto')
plt.colorbar()
plt.show()



### Functional Model ResNet

In [8]:
from keras.layers import Conv2D, Input, Dense
from keras.models import Model

def Residual(x, layers=32):
    conv = Conv2D(layers, (3, 3), padding='same')(x)
    return keras.layers.add([x, conv])

inputs = Input(shape=(32, 1024, 1))
# 3x3 conv with 3 output channels (same as input channels)
r0 = Residual(inputs, 32)
strided0 = Conv2D(32, (3, 3), strides=2)(r0)

r1 = Residual(strided0, 32)
strided1 = Conv2D(64, (3, 3), strides=2)(r1)

r2 = Residual(strided1, 64)
strided2 = Conv2D(64, (3, 3), strides=2)(r2)

# a layer instance is callable on a tensor, and returns a tensor
flat = Flatten()(strided2)
dense0 = Dense(64, activation='relu')(flat)
dense1 = Dense(64, activation='relu')(dense0)
drop0 = Dropout(0.5)(dense1)
predictions = Dense(2, activation='linear')(drop0)

model = Model(inputs=inputs, outputs=predictions)
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 1024, 1)  0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 32, 1024, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
add_1 (Add)                     (None, 32, 1024, 32) 0           input_1[0][0]                    
                                                                 conv2d_5[0][0]                   
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 15, 511, 32)  9248        add_1[0][0]                      
__________

In [ ]:
model_fn = dir_name + 'models/1sig/resnet.h5'
history_fn = dir_name + 'models/1sig/resnet_history'

epochs = 10

# Train
history = model.fit_generator(generator=training_generator,
                    steps_per_epoch=train_num // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=validation_num // batch_size,
                    callbacks=[keras.callbacks.ModelCheckpoint(model_fn, monitor='loss', verbose=0, save_best_only=True, mode='auto'), 
                               keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.0001), 
                               keras.callbacks.EarlyStopping(monitor='loss', patience=10,verbose=0, mode='auto')])

model.save_weights(model_fn)
with open(history_fn, 'wb') as f:
    pickle.dump(history.history, f)


In [75]:
%matplotlib inline
import matplotlib.pyplot as plt
loss = history.history['loss']
loss[3] = 0
# "Loss"
plt.plot(loss)
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

### Inception

In [10]:
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.layers import Conv2D, Input, Dense
from keras.models import Model

def Residual(x, layers=32):
    conv = Conv2D(layers, (3, 3), padding='same')(x)
    return keras.layers.add([x, conv])

def Inception(x, layers=64):
    tower_1 = Conv2D(layers, (1, 1), padding='same', activation='relu')(x)
    tower_1 = Conv2D(layers, (3, 3), padding='same', activation='relu')(tower_1)

    tower_2 = Conv2D(layers, (1, 1), padding='same', activation='relu')(x)
    tower_2 = Conv2D(layers, (5, 5), padding='same', activation='relu')(tower_2)

    tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    tower_3 = Conv2D(layers, (1, 1), padding='same', activation='relu')(tower_3)

    output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)
    return output

inputs = Input(shape=(32, 1024, 1))
# 3x3 conv with 3 output channels (same as input channels)
i0 = Inception(inputs, 32)
strided0 = Conv2D(32, (5, 3), strides=(1, 2))(i0)
c1 = Conv2D(32, (3, 3), strides=2)(strided0)

# r1 = Residual(strided0, 32)
# strided1 = Conv2D(64, (3, 3), strides=2)(r1)

# r2 = Residual(strided1, 64)
# strided2 = Conv2D(64, (3, 3), strides=2)(r2)

# a layer instance is callable on a tensor, and returns a tensor
flat = Flatten()(c1)
dense0 = Dense(32, activation='relu')(flat)
dense1 = Dense(64, activation='relu')(dense0)
drop0 = Dropout(0.5)(dense1)
predictions = Dense(2, activation='linear')(drop0)

model = Model(inputs=inputs, outputs=predictions)
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

from keras.utils import plot_model
plot_model(model, to_file='inception_model.png', show_shapes=False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 1024, 1)  0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 32, 1024, 32) 64          input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 32, 1024, 32) 64          input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 32, 1024, 1)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_12 

In [ ]:
model_fn = dir_name + 'models/1sig/inception.h5'
history_fn = dir_name + 'models/1sig/inception_history'

epochs = 10

# Train
history = model.fit_generator(generator=training_generator,
                    steps_per_epoch=train_num // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=validation_num // batch_size,
                    callbacks=[keras.callbacks.ModelCheckpoint(model_fn, monitor='loss', verbose=0, save_best_only=True, mode='auto'), 
                               keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.0001), 
                               keras.callbacks.EarlyStopping(monitor='loss', patience=10,verbose=0, mode='auto')])

model.save_weights(model_fn)
with open(history_fn, 'wb') as f:
    pickle.dump(history.history, f)

%matplotlib inline
import matplotlib.pyplot as plt
# "Loss"
plt.plot(loss = history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [27]:
list(range(1,2))

[1]